In [1]:
# !pip install ipython-autotime library to count run time
%load_ext autotime

time: 0 ns (started: 2021-09-16 15:16:24 +08:00)


In [2]:
import cv2
import os
from skimage.util import random_noise
from imwatermark import WatermarkEncoder
# from blind_watermark import WaterMark
from imwatermark import WatermarkDecoder
import numpy as np
import random
import glob
import pandas as pd 

time: 2.3 s (started: 2021-09-16 15:16:24 +08:00)


In [3]:
# this need to import seperately, else it will kernel die
from blind_watermark import WaterMark

bwm1 = WaterMark(password_img=1, password_wm=1)
bwm1.read_img('test/MRI_A.png')
wm = 'zhen'
bwm1.read_wm(wm, mode='str')

bwm1.embed('test/MRI_A.png')
len_wm = len(bwm1.wm_bit)
print('Put down the length of wm_bit {len_wm}'.format(len_wm=len_wm))

Put down the length of wm_bit 31
time: 2 s (started: 2021-09-16 15:16:26 +08:00)


# Part A: Encode

In [4]:
#Encode DWT A

infolder='in_img'
outfolder=os.path.join('out_img','algo_a')
in_img_list = os.listdir(infolder)
text='zhen'

for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_A_'+img
    out_path = os.path.join(outfolder,out_name)
    
   
    encoder = WaterMark(password_img=1, password_wm=1)
    encoder.read_img(in_path)
    encoder.read_wm(text, mode='str')
    encoder.embed(out_path)
     
    print("Done Encoding: " + out_path)

Done Encoding: out_img\algo_a\Algo_A_MRI_A.png
Done Encoding: out_img\algo_a\Algo_A_MRI_B.png
Done Encoding: out_img\algo_a\Algo_A_MRI_C.png
Done Encoding: out_img\algo_a\Algo_A_MRI_D.png
Done Encoding: out_img\algo_a\Algo_A_MRI_E.png
Done Encoding: out_img\algo_a\Algo_A_MRI_F.png
Done Encoding: out_img\algo_a\Algo_A_MRI_G.png


KeyboardInterrupt: 

time: 16.6 s (started: 2021-09-16 15:16:28 +08:00)


In [ ]:
#Encode dwtDct B

infolder='in_img'
outfolder=os.path.join('out_img','algo_b')
in_img_list = os.listdir(infolder)
text='zhen'

encoder = WatermarkEncoder()


for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_B_'+img
    out_path = os.path.join(outfolder,out_name)
    
    in_img = cv2.imread(in_path)
    
    encoder.set_watermark('bytes', text.encode('utf-8'))
    
    img_encoded = encoder.encode(in_img,'dwtDct')
    
    cv2.imwrite(out_path,img_encoded)
    
    print("Done Encoding: " + out_path)
    


In [ ]:
#encode DwtDctSvd C

infolder='in_img'
outfolder=os.path.join('out_img','algo_c')
in_img_list = os.listdir(infolder)
text='zhen'

encoder = WatermarkEncoder()

for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_C_'+img
    out_path = os.path.join(outfolder,out_name)
    
    in_img = cv2.imread(in_path)
    
    encoder.set_watermark('bytes', text.encode('utf-8'))
    
    img_encoded = encoder.encode(in_img,'dwtDctSvd')
    
    cv2.imwrite(out_path,img_encoded)
    
    print("Done Encoding: " + out_path)
    


In [ ]:
#encode RivaGAN D

infolder='in_img'
outfolder=os.path.join('out_img','algo_d')
in_img_list = os.listdir(infolder)
text='zhen'

encoder = WatermarkEncoder()

for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_D_'+img
    out_path = os.path.join(outfolder,out_name)
    
    in_img = cv2.imread(in_path)
    
    encoder.set_watermark('bytes', text.encode('utf-8'))
    encoder.loadModel()
    img_encoded = encoder.encode(in_img,'rivaGan')
    
    cv2.imwrite(out_path,img_encoded)
    
    print("Done Encoding: " + out_path)
    


# Part B: Attacking

In [7]:
#Attacking Methods

def gaussian(img, output_file_name):
    img_shape = img.shape
    gauss_img = random_noise(img, mode='gaussian', mean=0, var=0.01, clip= True)
    gauss_img = np.array(255*gauss_img, dtype = 'uint8')
    cv2.imwrite(output_file_name, gauss_img)

def salt_pepper(img, output_file_name):
    ratio = 0.1 #Probability of the noise
    #0.01 all pass?
    #0.1 algorithm d only partial pass
    snp_img = np.zeros(img.shape,np.uint8)
    thres = 1 - ratio
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            rdn = random.random()
            if rdn < ratio:
                snp_img[i][j] = 0
            elif rdn > thres:
                snp_img[i][j] = 255
            else:
                snp_img[i][j] = img[i][j]
    cv2.imwrite(output_file_name, snp_img)
    
def speckle(img, output_file_name):
    speckle_img = random_noise(img, mode='speckle', mean=0, var=0.01, clip= True)
    # The above function returns a floating-point image on the range [0, 1], thus we changed it to 'uint8' and from [0,255]
    speckle_img = np.array(255*speckle_img, dtype = 'uint8')
    cv2.imwrite(output_file_name, speckle_img)
    
def poisson(img, output_file_name):
    img_shape = img.shape
    noise = np.random.poisson(20, img_shape)
    #30 all fail 
    output = img + noise
    cv2.imwrite(output_file_name, output)
    
def rotate(img, output_file_name, angle=10):
    rows, cols, _ = img.shape
    M = cv2.getRotationMatrix2D(center=(cols / 2, rows / 2), angle=angle, scale=1)
    output_img = cv2.warpAffine(img, M, (cols, rows))
    cv2.imwrite(output_file_name, output_img)
    
def upscale(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.resize(img, None, fx=1.25, fy=1.25, interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(output_file_name, output_img)
    
def downscale(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.resize(img, None, fx=0.75, fy=0.75, interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(output_file_name, output_img)
    
def averaging(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.blur(img,(5,5)) 
    cv2.imwrite(output_file_name, output_img)
    
def gaussian_blurring(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.GaussianBlur(img,(5,5),0) 
    cv2.imwrite(output_file_name, output_img)
    
def median_blurring(img, output_file_name):
    output_img = cv2.medianBlur(img,7)
    #5 all pass
    cv2.imwrite(output_file_name, output_img)
    
def bilateral_filtering(img, output_file_name):
    output_img = cv2.bilateralFilter(img,9,75,75)
    cv2.imwrite(output_file_name, output_img)
    
def sharpen_filtering(img, output_file_name):
    sharpen_filter = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharped_img = cv2.filter2D(img, -1, sharpen_filter)
    cv2.imwrite(output_file_name, sharped_img)

def crop_horizontal(img, output_file_name):
    ratio = 0.5  
    #left 50% all pass
    img_shape = img.shape
    height = int(img_shape[0] * ratio)
    horizontal_img = img[:height, :, :]
    cv2.imwrite(output_file_name, horizontal_img)

def crop_vertical(img, output_file_name):
    ratio = 0.5
    img_shape = img.shape
    vertical = int(img_shape[1] * ratio)
    vertical_img = img[:, :vertical, :]
    cv2.imwrite(output_file_name, vertical_img)

def increase_brightness(img, output_file_name):
    ratio = 1.4 # 
    #+10% all pass
    #+50% algorithm D
    inc_bright_img = img * ratio
    inc_bright_img[inc_bright_img > 255] = 255
    cv2.imwrite(output_file_name, inc_bright_img)
    
def decrease_brightness(img, output_file_name):
    ratio = 0.6 
    # -50% brightness all pass
    dec_bright_img = img * ratio
    dec_bright_img[dec_bright_img > 255] = 255
    cv2.imwrite(output_file_name, dec_bright_img)
    
def masks(img, output_file_name):
    n = 5 # Set 2 mask
    ratio = 0.3 # mask ratio
    input_img_shape = img.shape
    mask_img = img.copy()
    for i in range(n):
        tmp = np.random.rand() * (1 - ratio)  # random one place to put mask，1-ratio to avoid overfloat
        start_height, end_height = int(tmp * input_img_shape[0]), int((tmp + ratio) * input_img_shape[0])
        tmp = np.random.rand() * (1 - ratio)
        start_width, end_width = int(tmp * input_img_shape[1]), int((tmp + ratio) * input_img_shape[1])

        mask_img[start_height:end_height, start_width:end_width, :] = 0
    cv2.imwrite(output_file_name, mask_img)
    
def init_attack(input_image, image_name):
    print('Attacking ' + image_name)
    
    a = image_name.split('.')[0]
    mri_name = a.split('_')[2]+'_'+a.split('_')[3]
    algo_name = a.split('_')[0]+'_'+a.split('_')[1]
    
    
    output_path=os.path.join('out_att',mri_name,algo_name.lower())
#     print(output_path)
    
    gaussian(input_image, os.path.join(output_path,'Gaussian_Noise.png'))
    salt_pepper(input_image, os.path.join(output_path,'Salt_Pepper.png'))
    speckle(input_image, os.path.join(output_path,'Speckle.png'))
    poisson(input_image, os.path.join(output_path,'Poisson.png'))
    rotate(input_image, os.path.join(output_path,'Rotate.png'), angle=10)
    upscale(input_image, os.path.join(output_path,'ScaleUp.png'))
    downscale(input_image, os.path.join(output_path,'ScaleDown.png'))
    averaging(input_image, os.path.join(output_path,'Averaging.png'))
    gaussian_blurring(input_image, os.path.join(output_path,'Gaussian_Blurring.png'))
    median_blurring(input_image, os.path.join(output_path,'Median_Blurring.png'))
    bilateral_filtering(input_image, os.path.join(output_path,'Bilateral_Filtering.png'))
    sharpen_filtering(input_image, os.path.join(output_path,'Sharpen_Filtering.png'))
    crop_horizontal(input_image, os.path.join(output_path,'Crop_Horizontal.png'))
    crop_vertical(input_image, os.path.join(output_path,'Crop_Vertical.png'))
    increase_brightness(input_image, os.path.join(output_path,'Brightness_Increase.png'))
    decrease_brightness(input_image, os.path.join(output_path,'Brightness_Decrease.png'))
    masks(input_image, os.path.join(output_path,'Masks.png'))
    cv2.imwrite(os.path.join(output_path,'JPG.jpg'), input_image)


time: 0 ns (started: 2021-09-16 15:17:34 +08:00)


In [8]:
infolder=os.path.join('out_img')
outfolder=os.path.join('out_att')

img_folder_list = os.listdir(infolder) 

for folder in img_folder_list: 
#     print(folder)
    in_img_list = os.listdir(os.path.join(infolder,folder)) 
    
    for img_path in in_img_list: 
#         print(img_path)
        img = cv2.imread(os.path.join(infolder,folder,img_path))
        init_attack(img,img_path)   
        

Attacking Algo_A_MRI_A.png
Attacking Algo_A_MRI_B.png
Attacking Algo_A_MRI_C.png
Attacking Algo_A_MRI_D.png
Attacking Algo_A_MRI_E.png
Attacking Algo_A_MRI_F.png
Attacking Algo_A_MRI_G.png
Attacking Algo_A_MRI_H.png
Attacking Algo_A_MRI_I.png
Attacking Algo_A_MRI_J.png
Attacking Algo_A_MRI_K.png
Attacking Algo_A_MRI_L.png
Attacking Algo_A_MRI_M.png
Attacking Algo_A_MRI_N.png
Attacking Algo_A_MRI_O.png
Attacking Algo_A_MRI_P.png
Attacking Algo_A_MRI_Q.png
Attacking Algo_A_MRI_R.png
Attacking Algo_A_MRI_S.png
Attacking Algo_A_MRI_T.png
Attacking Algo_B_MRI_A.png
Attacking Algo_B_MRI_B.png
Attacking Algo_B_MRI_C.png
Attacking Algo_B_MRI_D.png
Attacking Algo_B_MRI_E.png
Attacking Algo_B_MRI_F.png
Attacking Algo_B_MRI_G.png
Attacking Algo_B_MRI_H.png
Attacking Algo_B_MRI_I.png
Attacking Algo_B_MRI_J.png
Attacking Algo_B_MRI_K.png
Attacking Algo_B_MRI_L.png
Attacking Algo_B_MRI_M.png
Attacking Algo_B_MRI_N.png
Attacking Algo_B_MRI_O.png
Attacking Algo_B_MRI_P.png
Attacking Algo_B_MRI_Q.png
A

# Part C: Decode

In [10]:
def algo_A(image_list, result):
    count = 1
    print('ALGORITHM A')
    print('===========')
    for img in image_list:
        name = os.path.basename(img)
        name = name.split('.')[0]
        name = name.replace('_','')
        print(str(count) + ': ' + name)

        try:
            bwm1=WaterMark(password_img=1,password_wm=1)
            output = bwm1.extract(img, wm_shape=31, mode='str')
            print('Decoded text: ' + output)
            if output == 'zhen':
                print('Result      : Pass')
                result.append('Pass')
            else:
                print('Result      : Fail')
                result.append('Fail')
        except:
            print('Decoded text: Error')
            print('Result      : Fail')
            result.append('Fail')
            pass
        
        count += 1
        print('-----')

    
def algo_B(image_list, result):
    count = 1
    print('ALGORITHM B')
    print('===========')
    for img in image_list:
        bgr = cv2.imread(img)
        name = os.path.basename(img)
        name = name.split('.')[0]
        name = name.replace('_','')
        print(str(count) + ': ' + name)

        try:
            decoder = WatermarkDecoder('bytes', 32)
            watermark = decoder.decode(bgr, 'dwtDct')
            output = watermark.decode('utf-8')
            print('Decoded text: ' + output)
            if output == 'zhen':
                print('Result      : Pass')
                result.append('Pass')
            else:
                print('Result      : Fail')
                result.append('Fail')
        except:
            print('Decoded text: Error')
            print('Result      : Fail')
            result.append('Fail')
            pass
        
        count += 1
        print('-----')
        
def algo_C(image_list, result):
    count = 1
    print('ALGORITHM C')
    print('===========')
    for img in image_list:
        bgr = cv2.imread(img)
        name = os.path.basename(img)
        name = name.split('.')[0]
        name = name.replace('_','')
        print(str(count) + ': ' + name)

        try:
            decoder = WatermarkDecoder('bytes', 32)
            watermark = decoder.decode(bgr, 'dwtDctSvd')
            output = watermark.decode('utf-8')
            print('Decoded text: ' + output)
            if output == 'zhen':
                print('Result      : Pass')
                result.append('Pass')
            else:
                print('Result      : Fail')
                result.append('Fail')
        except:
            print('Decoded text: Error')
            print('Result      : Fail')
            result.append('Fail')
            pass
        
        count += 1
        print('-----')
        
def algo_D(image_list, result):
    count = 1
    print('ALGORITHM D')
    print('===========')
    for img in image_list:
        bgr = cv2.imread(img)
        name = os.path.basename(img)
        name = name.split('.')[0]
        name = name.replace('_','')
        print(str(count) + ': ' + name)

        try:
            decoder = WatermarkDecoder('bytes', 32)
            decoder.loadModel()
            watermark = decoder.decode(bgr, 'rivaGan')
            output = watermark.decode('utf-8')
            print('Decoded text: ' + output)
            if output == 'zhen':
                print('Result      : Pass')
                result.append('Pass')
            else:
                print('Result      : Fail')
                result.append('Fail')
        except:
            print('Decoded text: Error')
            print('Result      : Fail')
            result.append('Fail')
            pass
        
        count += 1
        print('-----')
        
              
                


time: 0 ns (started: 2021-09-16 15:19:14 +08:00)


In [11]:
def Decoder(df):
    img_folder_list = os.listdir('out_att') # Check all folder in this dic

    for folder in img_folder_list: 
        algo_list = os.listdir(os.path.join('out_att',folder)) # Check All algo folder
        
        for algo in algo_list: 
            imdir = os.path.join('out_att',folder,algo,'')
            ext = ['png', 'jpg'] 
            result = []
            result.append(folder)
            files = []
            [files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
            image_list = [(file) for file in files]

            if algo == 'algo_a':
                result.append(algo)
                print('[Photo: ' + folder + ']') # Show photo
                algo_A(image_list, result)
                print(result)
                df_length = len(df)
                df.loc[df_length] = result
                
            elif algo == 'algo_b':
                result.append(algo)
                print('[Photo: ' + folder + ']') # Show photo
                algo_B(image_list, result)            
                print(result)
                df_length = len(df)
                df.loc[df_length] = result
                
            elif algo == 'algo_c':
                result.append(algo)
                print('[Photo: ' + folder + ']') # Show photo
                algo_C(image_list, result)  
                print(result)
                df_length = len(df)
                df.loc[df_length] = result
                
            elif algo == 'algo_d':
                result.append(algo)
                print('[Photo: ' + folder + ']') # Show photo
                algo_D(image_list, result) 
                print(result)
                df_length = len(df)
                df.loc[df_length] = result  

time: 0 ns (started: 2021-09-16 15:19:14 +08:00)


In [14]:
#initiate decode

title = ['Photo', 'Algorithm', 'Averaging', 'BilateralFiltering', 'BrightnessDecrease',
        'BrightnessIncrease', 'CropHorizontal', 'CropVertical', 'GaussianBlurring', 
        'GaussianNoise', 'Masks', 'MedianBlurring', 'Poisson', 'Rotate', 'SaltPeper',
        'ScaleDown', 'ScaleUp', 'SharpenFiltering', 'Speckle','JPG']

df = pd.DataFrame(columns=title)

Decoder(df)

output_path=os.path.join('csv','result2.csv')

df.to_csv(output_path, index = False)

[Photo: MRI_A]
ALGORITHM A
1: Averaging
Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: zhen
Result      : Pass
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: Error
Result      : Fail
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: Error
Result      : Fail
-----
7: GaussianBlurring
Decoded text: zhen
Result      : Pass
-----
8: GaussianNoise
Decoded text:  @ 
Result      : Fail
-----
9: Masks
Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text: zhen
Result      : Pass
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text: Error
Result      : Fail
-----
13: SaltPepper
Decoded text: zhen
Result      : Pass
-----
14: ScaleDown
Decoded text: Error
Result      : Fail
-----
15: ScaleUp
Decoded text: Error
Result      : Fail
-----
16: SharpenFiltering
Decoded text: Error
Result      : Fail
-----


Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text: zhen
Result      : Pass
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text:     
Result      : Fail
-----
13: SaltPepper
Decoded text: zhen
Result      : Pass
-----
14: ScaleDown
Decoded text:     
Result      : Fail
-----
15: ScaleUp
Decoded text:    
Result      : Fail
-----
16: SharpenFiltering
Decoded text: Error
Result      : Fail
-----
17: Speckle
Decoded text: zhen
Result      : Pass
-----
18: JPG
Decoded text: zhen
Result      : Pass
-----
['MRI_B', 'algo_c', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Pass']
[Photo: MRI_B]
ALGORITHM D
1: Averaging
Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: zhen
Result      : Pass
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : 

Decoded text: zhen
Result      : Pass
-----
['MRI_D', 'algo_a', 'Pass', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Pass']
[Photo: MRI_D]
ALGORITHM B
1: Averaging
Decoded text:     
Result      : Fail
-----
2: BilateralFiltering
Decoded text:     
Result      : Fail
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : Pass
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: Error
Result      : Fail
-----
7: GaussianBlurring
Decoded text:     
Result      : Fail
-----
8: GaussianNoise
Decoded text: Error
Result      : Fail
-----
9: Masks
Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text:     
Result      : Fail
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text:     
Result      : Fail
-----
13: SaltPepper
Decoded text: zhen


Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: zhen
Result      : Pass
-----
7: GaussianBlurring
Decoded text: zhen
Result      : Pass
-----
8: GaussianNoise
Decoded text: zhen
Result      : Pass
-----
9: Masks
Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text: zhen
Result      : Pass
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text: z`en
Result      : Fail
-----
13: SaltPepper
Decoded text: xle&
Result      : Fail
-----
14: ScaleDown
Decoded text: zhen
Result      : Pass
-----
15: ScaleUp
Decoded text: zhen
Result      : Pass
-----
16: SharpenFiltering
Decoded text: zhen
Result      : Pass
-----
17: Speckle
Decoded text: zhen
Result      : Pass
-----
18: JPG
Decoded text: zhen
Result      : Pass
-----
['MRI_E', 'algo_d', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Fail', 'Fail', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass']
[Photo: MRI_F]
ALGORITHM A
1: Aver

Decoded text:     
Result      : Fail
-----
16: SharpenFiltering
Decoded text: Error
Result      : Fail
-----
17: Speckle
Decoded text: zhen
Result      : Pass
-----
18: JPG
Decoded text:     
Result      : Fail
-----
['MRI_G', 'algo_b', 'Fail', 'Fail', 'Pass', 'Pass', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Fail']
[Photo: MRI_G]
ALGORITHM C
1: Averaging
Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: xhen
Result      : Fail
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : Pass
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: Error
Result      : Fail
-----
7: GaussianBlurring
Decoded text: zhen
Result      : Pass
-----
8: GaussianNoise
Decoded text: Error
Result      : Fail
-----
9: Masks
Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text: z

...

Decoded text: z`en
Result      : Fail
-----
13: SaltPepper
Decoded text: h`%&
Result      : Fail
-----
14: ScaleDown
Decoded text: z`en
Result      : Fail
-----
15: ScaleUp
Decoded text: zhen
Result      : Pass
-----
16: SharpenFiltering
Decoded text: zhen
Result      : Pass
-----
17: Speckle
Decoded text: zhen
Result      : Pass
-----
18: JPG
Decoded text: zhen
Result      : Pass
-----
['MRI_M', 'algo_d', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Pass', 'Pass', 'Pass']
[Photo: MRI_N]
ALGORITHM A
1: Averaging
Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: zhen
Result      : Pass
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: Error
Result      : Fail
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: Error
Result      : Fail
-----
7: GaussianBlurring
Decoded text: zhen
R

Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: zhen
Result      : Pass
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : Pass
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: Error
Result      : Fail
-----
7: GaussianBlurring
Decoded text: zhen
Result      : Pass
-----
8: GaussianNoise
Decoded text: Error
Result      : Fail
-----
9: Masks
Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text: zhen
Result      : Pass
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text:     
Result      : Fail
-----
13: SaltPepper
Decoded text: zhen
Result      : Pass
-----
14: ScaleDown
Decoded text:     
Result      : Fail
-----
15: ScaleUp
Decoded text:     
Result      : Fail
-----
16: SharpenFiltering
Decoded text: Error
Result      : Fail
-----
17: Speckle
Decoded text: zhen
Result      

Decoded text: zhen
Result      : Pass
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text: Error
Result      : Fail
-----
13: SaltPepper
Decoded text: zhen
Result      : Pass
-----
14: ScaleDown
Decoded text: Error
Result      : Fail
-----
15: ScaleUp
Decoded text: Error
Result      : Fail
-----
16: SharpenFiltering
Decoded text: Error
Result      : Fail
-----
17: Speckle
Decoded text: zhen
Result      : Pass
-----
18: JPG
Decoded text: zhen
Result      : Pass
-----
['MRI_Q', 'algo_a', 'Pass', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Pass']
[Photo: MRI_Q]
ALGORITHM B
1: Averaging
Decoded text:     
Result      : Fail
-----
2: BilateralFiltering
Decoded text:     
Result      : Fail
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : Pass
-----
5: CropHorizontal
Decoded text: zhen
Result      

Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: zhen
Result      : Pass
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : Pass
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: zhen
Result      : Pass
-----
7: GaussianBlurring
Decoded text: zhen
Result      : Pass
-----
8: GaussianNoise
Decoded text: zhen
Result      : Pass
-----
9: Masks
Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text: zhen
Result      : Pass
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text: z`en
Result      : Fail
-----
13: SaltPepper
Decoded text: zle&
Result      : Fail
-----
14: ScaleDown
Decoded text: zhen
Result      : Pass
-----
15: ScaleUp
Decoded text: zhen
Result      : Pass
-----
16: SharpenFiltering
Decoded text: zhen
Result      : Pass
-----
17: Speckle
Decoded text: zhen
Result      : P

Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text:     
Result      : Fail
-----
13: SaltPepper
Decoded text: zhen
Result      : Pass
-----
14: ScaleDown
Decoded text:     
Result      : Fail
-----
15: ScaleUp
Decoded text: 
Result      : Fail
-----
16: SharpenFiltering
Decoded text: Error
Result      : Fail
-----
17: Speckle
Decoded text: zhen
Result      : Pass
-----
18: JPG
Decoded text:   
Result      : Fail
-----
['MRI_T', 'algo_b', 'Fail', 'Fail', 'Pass', 'Pass', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Fail']
[Photo: MRI_T]
ALGORITHM C
1: Averaging
Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: zhen
Result      : Pass
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : Pass
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: zhzh
Result      :

In [15]:
df

,Photo,Algorithm,Averaging,BilateralFiltering,BrightnessDecrease,BrightnessIncrease,CropHorizontal,CropVertical,GaussianBlurring,GaussianNoise,Masks,MedianBlurring,Poisson,Rotate,SaltPeper,ScaleDown,ScaleUp,SharpenFiltering,Speckle,JPG
0,MRI_A,algo_a,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Pass
1,MRI_A,algo_b,Fail,Fail,Pass,Pass,Pass,Fail,Fail,Fail,Pass,Fail,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Fail
2,MRI_A,algo_c,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Pass
3,MRI_A,algo_d,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Fail,Fail,Pass,Pass,Pass,Pass,Pass
4,MRI_B,algo_a,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Fail,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,MRI_S,algo_d,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Fail,Fail,Pass,Pass,Pass,Pass,Pass
76,MRI_T,algo_a,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Pass
77,MRI_T,algo_b,Fail,Fail,Pass,Pass,Pass,Fail,Fail,Fail,Pass,Fail,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Fail
78,MRI_T,algo_c,Pass,Pass,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Pass


time: 62 ms (started: 2021-09-16 15:49:10 +08:00)


# TODO:

>PSNR NCC